In [1]:
%pip install xarray numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
import xarray as xr
import numpy as np

ds = xr.open_dataset('./Land_and_Ocean_LatLong1.nc')

In [5]:
print(ds)

<xarray.Dataset> Size: 545MB
Dimensions:      (longitude: 360, latitude: 180, time: 2089, month_number: 12)
Coordinates:
  * longitude    (longitude) float32 1kB -179.5 -178.5 -177.5 ... 178.5 179.5
  * latitude     (latitude) float32 720B -89.5 -88.5 -87.5 ... 87.5 88.5 89.5
  * time         (time) float64 17kB 1.85e+03 1.85e+03 ... 2.024e+03 2.024e+03
Dimensions without coordinates: month_number
Data variables:
    land_mask    (latitude, longitude) float64 518kB ...
    temperature  (time, latitude, longitude) float32 541MB ...
    climatology  (month_number, latitude, longitude) float32 3MB ...
Attributes:
    Conventions:           Berkeley Earth Internal Convention (based on CF-1.5)
    title:                 Native Format Berkeley Earth Surface Temperature A...
    history:               20-Feb-2024 10:04:52
    institution:           Berkeley Earth Surface Temperature Project
    land_source_history:   07-Feb-2024 05:19:02
    ocean_source_history:  20-Feb-2024 08:20:00
    com

In [6]:
# Change the time dimension to be a simple index 
# Time: 1850 Jan = 0, 1850 Feb = 1, etc.
# Longitude: 90S to 90N, 0 to 179
# Latitude: 180W to 180E, 0 to 359
time_indices = np.arange(len(ds['time']))
lat_indices = np.arange(len(ds['latitude']))
long_indices = np.arange(len(ds['longitude']))

ds = ds.assign_coords(time = time_indices, latitude=lat_indices, longitude=long_indices)


In [11]:
def index_to_date(index):
    base_year = 1850
    year = base_year + index // 12
    month = index % 12 + 1  # January is 1, February is 2, etc.
    return f"{year}-{month:02d}"

def index_to_lat(index):
    return index - 89.5

def index_to_long(index):
    return index - 189.5

def date_to_index(date):
    year, month = date.split('-')
    return (int(year) - 1850) * 12 + int(month) - 1

def lat_to_index(lat):
    # Latitude ranges from -89.5 to 89.5 with a step of 1
    return int(lat + 89.5)

def long_to_index(long):
    # Longitude ranges from -189.5 to 189.5 with a step of 1
    return int(long + 189.5)


In [13]:
time_index = date_to_index('1940-04')  # For the second time point
lat_index = lat_to_index(0)  # Convert latitude 0 to its corresponding index
long_index = long_to_index(0)  # Convert longitude 0 to its corresponding index

temperature_data = ds['temperature'].isel(time=time_index, latitude=lat_index, longitude=long_index)

print(temperature_data.values)

0.19414559
